In [0]:
from re import search
from sqlalchemy import create_engine

import duckdb
import pandas as pd 
import numpy as np 

import mysql.connector

import warnings
warnings.filterwarnings('ignore')

from custom_packages.email_operations import EmailOperations

# pd.options.display.max_rows = None

pd.set_option('display.float_format', lambda x: '%.2f' % x)

import json
with open('/Workspace/Credentials/db_data.json', 'r') as fp:
    data = json.load(fp)

Dumpdb = mysql.connector.connect(
  host=data['Dumpdb']['host'],
  user=data['Dumpdb']['user'],
  passwd=data['Dumpdb']['passwd'],
  database = data['Dumpdb']['database']
)


Mifosdb = mysql.connector.connect(
  host=data['mifos']['host'],
  user=data['mifos']['user'],
  passwd=data['mifos']['passwd'],
  database = data['mifos']['database']
)


vbizdb = mysql.connector.connect(
     host=data['vbizdb']['host'],
  user=data['vbizdb']['user'],
  passwd=data['vbizdb']['passwd'],
  database = data['vbizdb']['database']
)


host = data['redshift']['host']
user = data['redshift']['user']
passwd = data['redshift']['passwd']
database = data['redshift']['database']

conn = create_engine(f"postgresql+psycopg2://{user}:{passwd}@{host}:5439/{database}")
pd.set_option('display.float_format', lambda x: '%.2f' % x)
# , fast_executemany=True)

In [0]:
from datetime import datetime, timedelta

today = datetime.today().strftime('%Y-%m-%d')
print(today)

In [0]:
%%time
def get_referrals(referral_code):
    rf = pd.read_sql_query(f'''select referral_id, mc.id, display_name, sa.account_no, sa.product_id, sp.name from m_client mc 
    left join m_savings_account sa on sa.client_id = mc.id
    left join m_savings_product sp on sp.id = sa.product_id
    where referral_id = '{referral_code}'
    ''', Mifosdb)
    
    referred_by_id = rf['id'][0]
    
    mif = f'''select mc.id as client_id, mc.display_name as client_name, sa.account_no, sa.id as account_id, sa.product_id, sp.name as product_name, 
    sa.account_balance_derived, sa.approvedon_date as activation_date,
    sa.closedon_date, referred_by_id
    from m_client mc 
    join m_savings_account sa on sa.client_id = mc.id 
    join m_savings_product sp on sp.id = sa.product_id
    where mc.referred_by_id = '{referred_by_id}'
    and isnull(sa.closedon_date)
    -- AND LEFT(sa.approvedon_date, 7) >= '2023-01'
    order by 7 desc
    '''
    
    dfmif = pd.read_sql_query(mif, Mifosdb)
    
    
    lo = f'''select mc.display_name as client_name, sa.account_no, sa.id as loan_id, sa.product_id, sp.name as product_name, 
    sa.principal_disbursed_derived as principal_disbursed, sa.principal_outstanding_derived as principal_outstanding, sa.disbursedon_date as disbursement_date,
    sa.closedon_date, referred_by_id
    from m_client mc 
    left join m_loan sa on sa.client_id = mc.id 
    join m_product_loan sp on sp.id = sa.product_id
    where mc.referred_by_id = '{referred_by_id}'
    -- and left(sa.disbursedon_date, 7) >= '2023-01'
    -- and isnull(sa.closedon_date)
    -- and !isnull(sa.disbursedon_date)
    order by 6 desc
    '''
    
    dflo = pd.read_sql_query(lo, Mifosdb)
    
    return rf, dfmif, dflo

df = get_referrals('FSNH4') #('TVSKX') # , XAXFW, KB5QB, DTNVG') #, KB5QB, DTNVG') #TVSKX  5PL7O WK12A k7m4e
ref = df[0]['referral_id'][0]

In [0]:
# with pd.ExcelWriter(f"C:\\Users\\babatunde.omotayo\\Downloads\\V_bank_tasks\\Raphael downlines\\{ref} accounts as at {today}.xlsx") as writer:
#     df[1].to_excel(writer, sheet_name = 'deposits', index = False)
#     df[2].to_excel(writer, sheet_name = 'loans', index = False)


In [0]:
staff_list = [('Raphael', '5PL7O', 'raphael.oluwole@vfd-mfb.com'), 
              ('Joshua', 'K7M4E', 'joshua.vincent@vfd-mfb.com'),
             ('Simisola', 'VGFAZ',  'simisola.olanrewaju@vfd-mfb.com'),
              ('Vivian', 'FSNH4', 'vivian.ezeani@vfd-mfb.com')]
             #('Tobi', '3PZKO', 'oluwatobi.odusanya@vfd-mfb.com'),
             

In [0]:
staff_list.extend([
    ('Jonathan', 'VZJT2', 'Jonathan.Ozoekwem@vfd-mfb.com'),
    ('Nosa', 'MUJOX', 'Nosa.Osemwegie@vfd-mfb.com'),
    ('Patrick', 'JWZ8P', 'Patrick.Chukwurah@vfd-mfb.com'),
    ('Simisola', 'VGFAZ', 'Simisola.Olanrewaju@vfd-mfb.com'),
    ('Elizabeth', 'SEDFI', 'Elizabeth.Enifeni@vfd-mfb.com'),
    ('Catherine', 'ZXMH5', 'Catherine.Onelum@vfd-mfb.com'),
    ('Raphael', 'KKPBV', 'Raphael.Egbe@vfd-mfb.com'),
    ('Yonodu', 'MLMD8', 'Yonodu.Okeugo@vfd-mfb.com'),
    ('Stephen', '3NDR5', 'Stephen.Tewogbade@vfd-mfb.com'),
    ('Mayowa', 'WK12A', 'Mayowa.Francis@vfd-mfb.com'),
    ('Vivian', 'FSNH4', 'Vivian.Ezeani@vfd-mfb.com'),
     # ('Ibrahim', 'DPHJS', 'Ibrahim.Ajiboye@vfd-mfb.com'),
    ('Blessing', 'TVSKX', 'Blessing.Idogei@vfd-mfb.com')
   
   
])


In [0]:
staff_list

In [0]:
%%time
for staff in staff_list[:]:
    df = get_referrals(staff[1]) #
    ref = df[0]['referral_id'][0]
    
    bal = df[1]
    ods = bal[bal['account_balance_derived'] < 0]
    deps = bal[bal['account_balance_derived'] >= 0]
    loans = df[2]
    with pd.ExcelWriter(f"C:\\Users\\babatunde.omotayo\\Downloads\\V_bank_tasks\\Raphael downlines\\{ref} accounts as at {today}.xlsx") as writer:
        deps.to_excel(writer, sheet_name = 'deposits', index = False)
        loans.to_excel(writer, sheet_name = 'loans', index = False)
        ods.to_excel(writer, sheet_name = 'overdrafts', index = False)
        
    # Initialize email object and its parameters

    email = EmailOperations

    filenames = [f"{ref} accounts as at {today}.xlsx"]

    SourcePathName  = f"C:\\Users\\babatunde.omotayo\\Downloads\\V_bank_tasks\\Raphael downlines\\"  

    subject =   f'Referral_id - {ref} Downline Accounts'

    name = staff[0]

    body = f'Your downline account deposits and loans is attached for your review.'

    recipient = staff[2]

    ## Send mail
    email.send_mail(filenames, SourcePathName, today, recipient, subject, name, body)

In [0]:
deps